In [1]:
import pandas as pd
import numpy as np
import pathlib
import datetime
import requests
import os

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'


def convert_path(file_path):
    fp = pathlib.PureWindowsPath(file_path).as_posix()

    return fp


def clean_units(df, col_list):
    for col in col_list:
        if col in list(df.columns):
            df[col] = df[col].str.replace('< 1', '1')
            df[col] = df[col].str.replace('<', '1')
            df[f"{col}_values"] = df[col].str.split(' ').str[0].astype(float)
            df[f"{col}_units"] = df[col].str.split(' ').str[1]

    return df


def nutrition_calcs(df, units='g', carb_ratio=4, protein_ratio=4,fat_ratio=9):
    
    df['protein_check'] = np.where(df['Protein_units'] == units, 0, 1)
    df['carb_check'] = np.where(df['Total Carbohydrate_units'] == units, 0, 1)
    df['fat_check'] = np.where(df['Total Fat_units'] == units, 0, 1)
    df['carbs_calories'] = df['Total Carbohydrate_values'] * carb_ratio
    df['protein_calories'] = df['Protein_values'] * protein_ratio
    df['fat_calories'] = df['Total Fat_values'] * fat_ratio

    df['f_c_p'] = df['carbs_calories'] + df['protein_calories'] + df['fat_calories']
    df['pct_fat'] = df['fat_calories'] / df['f_c_p']
    df['pct_carbs'] = df['carbs_calories'] / df['f_c_p']
    df['pct_protein'] = df['protein_calories'] / df['f_c_p']

    return df


def indicator_calcs(df, df_col, nutrition_ratio,variance_lower=0.05,variance_higher=0.05):
    variance_higher_calcs = (nutrition_ratio + variance_higher)
    variance_lower_calcs = (nutrition_ratio - variance_lower)

    return np.where(df[df_col].between(variance_lower_calcs,variance_higher_calcs,inclusive=True),1,0)

food_data_path = convert_path(r"C:\Users\CDT - Admin\PycharmProjects\AP_Apple_Meal_Recommendations\CalorieKing_Dataset_Cleaned_20210128 (1).csv")

# List select columns that we want to clean
col_list = ['Alcohol','Calcium','Cholesterol','Dietary Fiber','Iron',
            'Protein','Saturated Fat','Sodium','Sugars','Total Carbohydrate',
            'Total Fat','Vitamin A','Vitamin C','Potassium','Trans Fat','Monounsaturated Fat',
           'Polyunsaturated Fat','Omega- Fatty Acids','Caffeine']

food_df = pd.read_csv(food_data_path)

# Clean the units from the data and add the cleaned columns to the dataset so we can work with the numeric values
food_df = clean_units(food_df,col_list)

food_df.shape
food_df.head()


,item_name,Alcohol,Calcium,Calories,Cholesterol,Dietary Fiber,Iron,Protein,Saturated Fat,Sodium,...,Trans Fat_values,Trans Fat_units,Monounsaturated Fat_values,Monounsaturated Fat_units,Polyunsaturated Fat_values,Polyunsaturated Fat_units,Caffeine_values,Caffeine_units,Omega- Fatty Acids_values,Omega- Fatty Acids_units
0,Homestyle Bakes Cheesy Chicken Alfredo,0 g,150 mg,410,35 mg,3 g,3 mg,15 g,7 g,880 mg,...,0.0,g,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Chicken Fingers Dinner, frozen",0 g,80 mg,479,50 mg,5 g,3 mg,17 g,4.5 g,700 mg,...,0.0,g,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Original Fried Chicken, frozen",0 g,20 mg,330,75 mg,0.5 g,1 mg,24 g,5 g,890 mg,...,1.5,g,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Vegetarian Cheesy Bean & Rice Burrito,NaN,150 mg,410,5 mg,7 g,4 mg,10 g,4 g,880 mg,...,0.0,g,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Green Goddess Cobb Salad, with Chicken & Green...",0 g,NaN,550,290 mg,7 g,NaN,42 g,8 g,720 mg,...,0.0,g,NaN,NaN,NaN,NaN,0.0,mg,NaN,NaN


In [2]:
item_addition_df = pd.read_csv(convert_path(r"C:\Users\CDT - Admin\Downloads\Item_addition_20210322.csv"))
item_addition_df = clean_units(item_addition_df,col_list)

food_df = pd.concat([food_df,item_addition_df],axis=0)

In [3]:
food_df.to_csv('CalorieKing_Data_Cleaned.csv',index=False)

In [4]:
nutr_df = nutrition_calcs(df=food_df)

nutr_df['fat_indicator'] = indicator_calcs(df=nutr_df,df_col='pct_fat',nutrition_ratio=0.30)
nutr_df['carbs_indicator'] = indicator_calcs(df=nutr_df,df_col='pct_carbs',nutrition_ratio=0.55)
nutr_df['protein_indicator'] = indicator_calcs(df=nutr_df,df_col='pct_protein',nutrition_ratio=0.15)
nutr_df['indicator_sum'] = np.sum(nutr_df[['fat_indicator','carbs_indicator','protein_indicator']],axis=1)

nutr_df.to_csv('ap_apple_nutrition_calculations.csv',index=False)

In [5]:
# Filter the data so it only shows meal options that meet all 3 criteria that were defined in the previous cell.

meal_options_df = nutr_df.loc[nutr_df['indicator_sum'] == 3]

# Create a new dataset that contains data for select columns that are needed so it is easier to work with
meal_columns = ['Brand Name','Meal Type','item_name',
                'Calories','Protein_values','Total Fat_values','Total Carbohydrate_values',
                'carbs_calories','protein_calories','fat_calories','f_c_p',
                'pct_fat','pct_carbs','pct_protein','Source URL']

meal_recs_df = meal_options_df[meal_columns]
meal_recs_df

#meal_recs_df.to_csv('CriteriaMatch_All.csv',index=False)

# CREATE A NON-FILTERED VERSION OF THE ABOVE DATASET
#meal_recs_df2 = nutr_df[meal_columns]
#meal_recs_df2.to_csv('Unfiltered_Recs_with_calcs.csv',index=False)

,Brand Name,Meal Type,item_name,Calories,Protein_values,Total Fat_values,Total Carbohydrate_values,carbs_calories,protein_calories,fat_calories,f_c_p,pct_fat,pct_carbs,pct_protein,Source URL
59,Kroger,Energy/Nutritional Drinks,Fortify Plus Vanilla Nutrition Shake,350,13.0,11.0,50.0,200.0,52.0,99.0,351.0,0.282051,0.569801,0.148148,https://www.calorieking.com/us/en/foods/f/calo...
493,Food Lion,Soups,"Chicken & Stars Condensed Soup, canned",70,3.0,2.0,10.0,40.0,12.0,18.0,70.0,0.257143,0.571429,0.171429,https://www.calorieking.com/us/en/foods/f/calo...
504,Food Lion,Frozen Meals,"Chicken Sandwich, Hot & Spicy",420,18.0,15.0,53.0,212.0,72.0,135.0,419.0,0.322196,0.505967,0.171838,https://www.calorieking.com/us/en/foods/f/calo...
506,Food Lion,Frozen Pizzas,"Bake & Rise Crust Pizza, Pepperoni",320,13.0,12.0,40.0,160.0,52.0,108.0,320.0,0.337500,0.500000,0.162500,https://www.calorieking.com/us/en/foods/f/calo...
517,Food Lion,Soups,Condensed Chicken Noodle Soup,60,3.0,2.0,9.0,36.0,12.0,18.0,66.0,0.272727,0.545455,0.181818,https://www.calorieking.com/us/en/foods/f/calo...
521,Food Lion,Packaged Meals,Deluxe Shells & Cheddar Dinner With Cheddar Ch...,370,11.0,13.0,52.0,208.0,44.0,117.0,369.0,0.317073,0.563686,0.119241,https://www.calorieking.com/us/en/foods/f/calo...
523,Food Lion,Frozen Pizzas,Microwaveable Crispy Thin Crust Pizza,360,13.0,13.0,49.0,196.0,52.0,117.0,365.0,0.320548,0.536986,0.142466,https://www.calorieking.com/us/en/foods/f/calo...
539,Food Lion,Milk & Flavored Milk,Chocolate Milk,210,8.0,8.0,27.0,108.0,32.0,72.0,212.0,0.339623,0.509434,0.150943,https://www.calorieking.com/us/en/foods/f/calo...
541,Food Lion,Frozen Pizzas,Supreme Crispy Thin Crust Pizza,350,10.0,13.0,47.0,188.0,40.0,117.0,345.0,0.339130,0.544928,0.115942,https://www.calorieking.com/us/en/foods/f/calo...
543,Food Lion,"Breakfast Cereals, Ready To Eat","Granola, Old-Fashioned Rolled Oats, Protein Oa...",420,20.0,12.0,60.0,240.0,80.0,108.0,428.0,0.252336,0.560748,0.186916,https://www.calorieking.com/us/en/foods/f/calo...


In [8]:
frozen_meals_df = meal_recs_df.loc[meal_recs_df['Meal Type'] == 'Frozen Meals']
#frozen_meals_df = meal_recs_df
calories_input = 468
filter_bound = 1.20

meal_recs_filtered = frozen_meals_df.loc[(frozen_meals_df['Calories'].between(calories_input/filter_bound,
                                                                        calories_input*filter_bound,
                                                                        inclusive=True))]
meal_recs_filtered
print(len(meal_recs_filtered))
#meal_recs_filtered.to_csv('75105_meal_recs_2.csv',index=False)

,Brand Name,Meal Type,item_name,Calories,Protein_values,Total Fat_values,Total Carbohydrate_values,carbs_calories,protein_calories,fat_calories,f_c_p,pct_fat,pct_carbs,pct_protein,Source URL
504,Food Lion,Frozen Meals,"Chicken Sandwich, Hot & Spicy",420,18.0,15.0,53.0,212.0,72.0,135.0,419.0,0.322196,0.505967,0.171838,https://www.calorieking.com/us/en/foods/f/calo...
605,Food Lion,Frozen Meals,"Breaded Chicken Sandwich, frozen",410,18.0,13.0,54.0,216.0,72.0,117.0,405.0,0.288889,0.533333,0.177778,https://www.calorieking.com/us/en/foods/f/calo...
862,Banquet,Frozen Meals,"Sweet & Sour Chicken, frozen",390,10.0,14.0,56.0,224.0,40.0,126.0,390.0,0.323077,0.574359,0.102564,https://www.calorieking.com/us/en/foods/f/calo...
871,EVOL,Frozen Meals,"Egg & Potato Burrito, frozen",440,17.0,13.0,63.0,252.0,68.0,117.0,437.0,0.267735,0.576659,0.155606,https://www.calorieking.com/us/en/foods/f/calo...


4


In [18]:
#meal_recs_filtered.to_csv('75106_meal_recs.csv',index=False)

In [9]:
meal_recs_df.loc[meal_recs_df['Brand Name'] == 'Kroger']

,Brand Name,Meal Type,item_name,Calories,Protein_values,Total Fat_values,Total Carbohydrate_values,carbs_calories,protein_calories,fat_calories,f_c_p,pct_fat,pct_carbs,pct_protein,Source URL
59,Kroger,Energy/Nutritional Drinks,Fortify Plus Vanilla Nutrition Shake,350,13.0,11.0,50.0,200.0,52.0,99.0,351.0,0.282051,0.569801,0.148148,https://www.calorieking.com/us/en/foods/f/calo...


In [10]:
meal_options_df['Brand Name'].unique()

array(['Kroger', 'Food Lion', "Amy's", 'Banquet', 'EVOL', 'Devour',
       'Macaroni & Cheese (Kraft)', 'Lean Cuisine', "Trader Joe's",
       'Oscar Mayer', 'Birds Eye'], dtype=object)